In [1]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('train.csv')
train_df = train_df.drop(axis=1, columns=['ID'])
train_df.head()

test_df = pd.read_csv('test.csv')

In [3]:
X_train = train_df['Text']
Y_train = train_df['Class']

In [5]:
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
lemmatizer=WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/krishna_primary/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/krishna_primary/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
def preprocess(X):
  corpus = []
  for review in X:
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

  return corpus

In [7]:
X_train = preprocess(X_train)

['please enable removing language code dev center language history example ever selected ru ru ru laguages published xap store cause tile localization show en u default tile localization bad', 'note csproj file supportedcultures entry like supportedcultures de de ru ru ru supportedcultures removed ru language code published new xap version old xap version still remains store replaced unpublished', 'wich mean new version fully replaced old version cause serious problem', 'user still receive old xap version app', 'store randomly give old xap new xap version app', 'app wp version wp version xap submission', 'wp xap work wp wp xap work wp', 'sometimes store give wrong wp xap version app window phone user', 'option remove ru language code app language history option completly replace fully replace prevoius e xap', 'currently ever mistakenly selected ru language bad situation forever live wrong tile displayed russian language store english one publish update removed ru language update fully 

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [10]:
from numpy import asarray

In [11]:
MAX_NB_WORDS = 8000
MAX_SEQUENCE_LENGTH = 200

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

vocab_size = len(word_index)+1

In [12]:
X = tokenizer.texts_to_sequences(X_train)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of desc tensor:', X.shape)

Y = pd.get_dummies(Y_train).values
print('Shape of rating tensor:', Y.shape)

Shape of desc tensor: (8500, 200)
Shape of rating tensor: (8500, 2)


In [13]:
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [14]:
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.25, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(6375, 200) (6375, 2)
(2125, 200) (2125, 2)


In [16]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, GRU

np.random.seed(1)
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=200, trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(16, dropout=0.3, recurrent_dropout=0.3, use_bias=True))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer="nadam", metrics=['accuracy'])
model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 100)          808500    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 200, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 16)                7488      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                1088      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
Total params: 817,206
Trainable param

In [17]:
history = model.fit(X_train, Y_train, epochs=60, batch_size=64, validation_split=0.2, shuffle=True)

Train on 5100 samples, validate on 1275 samples
Epoch 1/60
5100/5100 [==============================] - 14s 3ms/step - loss: 0.5717 - acc: 0.7525 - val_loss: 0.5557 - val_acc: 0.7522
Epoch 2/60
5100/5100 [==============================] - 12s 2ms/step - loss: 0.5395 - acc: 0.7529 - val_loss: 0.4958 - val_acc: 0.7522
Epoch 3/60
5100/5100 [==============================] - 11s 2ms/step - loss: 0.5077 - acc: 0.7533 - val_loss: 0.4915 - val_acc: 0.7286
Epoch 4/60
5100/5100 [==============================] - 10s 2ms/step - loss: 0.4991 - acc: 0.7508 - val_loss: 0.4729 - val_acc: 0.7529
Epoch 5/60
5100/5100 [==============================] - 10s 2ms/step - loss: 0.4790 - acc: 0.7604 - val_loss: 0.4515 - val_acc: 0.7702
Epoch 6/60
5100/5100 [==============================] - 10s 2ms/step - loss: 0.4695 - acc: 0.7686 - val_loss: 0.4393 - val_acc: 0.7914
Epoch 7/60
5100/5100 [==============================] - 10s 2ms/step - loss: 0.4606 - acc: 0.7737 - val_loss: 0.4365 - val_acc: 0.7914
Epoch 8

In [21]:
model.save("model-2017A7PS0007G.h5")